# References

* [Charlie Morris ZL2CTM : 40m CW Transceiver - Part 2a: Oscillator Design](https://youtu.be/jUYMaFXl09s)
* [Solid State Design for the Radio Amateur, Wes Hayward W7ZOI (aka SSDRA)](https://archive.org/details/SolidStateDesignForTheRadioAmateur1986)

# Implementation Notes from SSDRA
* Limit the feedback ratio to what is strictly necessary to sustain oscillation. Excessive feedback can lead to distortion and unwanted harmonics.
* Bias the transistor at low current levels to minimize internal heating and improve frequency stability.
* Use high-Q components in the tank circuit.  Air-core inductors are preferred.
* Transistor with beta of >10 at the operating frequency is recommended.
* Keep ground planes away to minimize parasitic capacitances.
* Put oscillator in a shielded enclosure to minimize interference.
* Clapp oscillator can be easily created from Colpitts.  It has the advantage that larger inductor values can be used, reducing effect of stray inductances.

# Oscillator stage

In [1]:
# Transistor biasing
import math

Ie = 0.5e-3  # Emitter current in Amperes
Ve = 2.5  # Emitter voltage in Volts
Rc = 47 # Collector resistance in Ohms, for damping
Vcc = 5
Vbe = 0.7  # Base-Emitter voltage in Volts
hfe = 100  # Current gain

Re = Ve / Ie  # Emitter resistance in Ohms
print(f"Emitter Resistance (Re): {Re:.2f} Ohms")

# Voltage over R1
Vr1 = Vcc - Ie * Rc - Vbe - Ve
Ir1 = Ie / hfe * 11
R1 = Vr1 / Ir1  # Resistance R1 in Ohms
print(f"Resistance R1: {R1:.2f} Ohms")

# Voltage over R2
Vr2 = Ve + Vbe
Ir2 = Ie / hfe * 10
R2 = Vr2 / Ir2  # Resistance R2 in Ohms
print(f"Resistance R2: {R2:.2f} Ohms")

Emitter Resistance (Re): 5000.00 Ohms
Resistance R1: 32300.00 Ohms
Resistance R2: 64000.00 Ohms


So let's choose R1 = 33kΩ and R2 = 62kΩ.

In [2]:
# Tank circuit calculations
f = 98e6 - 10.7e6 # Center frequency for the oscillator in Hz
f_min= f - 10e6  # Minimum frequency in Hz
f_max= f + 10e6  # Maximum frequency in Hz

def calc_capacitance(Z, f):
    return 1 / (2 * math.pi * f * Z)

Xcf = 45  # Capacitive reactance in Ohms
Cf = calc_capacitance(Xcf, f)  # Capacitance in Farads
print(f"Capacitance Cf: {Cf * 1e12:.2f} pF")

Xcc = 100  # Capacitive reactance in Ohms
Cc = calc_capacitance(Xcc, f)  # Capacitance in Farads
print(f"Capacitance Cc: {Cc * 1e12:.2f} pF")

Xco = 750  # Capacitive reactance in Ohms
Co = calc_capacitance(Xco, f)  # Capacitance in Farads
print(f"Capacitance Co: {Co * 1e12:.2f} pF")

# Equivalent capacitance of Cc and Cf in series
Ceq = 1 / (1 / Cc + 2 / Cf)
print(f"Equivalent Capacitance (Ceq): {Ceq * 1e12:.2f} pF")

# Maximum value for the inductor (given no tuning capacitor is used)
L_max = 1 / (math.pow(2 * math.pi * f_max, 2) * Ceq)
print(f"Maximum Inductance (L_max): {L_max * 1e9:.2f} nH")

# Inductance calculation
XL1 = 93  # Inductive reactance in Ohms (original value = 140)
L1 = XL1 / (2 * math.pi * f)  # Inductance in Henries
print(f"Inductance L1: {L1 * 1e9:.2f} nH")

# Tuning range calculations
C_total_min = 1 / (math.pow(2 * math.pi * f_max, 2) * L1)
print(f"\nTotal Capacitance at f_max: {C_total_min * 1e12:.2f} pF")
C_total_max = 1 / (math.pow(2 * math.pi * f_min, 2) * L1)
print(f"Total Capacitance at f_min: {C_total_max * 1e12:.2f} pF")

Cvar_min = C_total_min - Ceq
Cvar_max = C_total_max - Ceq
print(f"Tuning Capacitor Range: {Cvar_min * 1e12:.2f} pF to {Cvar_max * 1e12:.2f} pF")

Capacitance Cf: 40.51 pF
Capacitance Cc: 18.23 pF
Capacitance Co: 2.43 pF
Equivalent Capacitance (Ceq): 9.60 pF
Maximum Inductance (L_max): 278.84 nH
Inductance L1: 169.55 nH

Total Capacitance at f_max: 15.78 pF
Total Capacitance at f_min: 25.00 pF
Tuning Capacitor Range: 6.19 pF to 15.41 pF


Original configuration : Cf = 39pF, Cc=18pF and Co=2pF and L1 = 240nH (Wuerth WE-CAIR)

With the original XL1 of 140ohm, we arrive at a tuning range for the capacitor of 1pF to 7pF.  This is too low to be realistic.  The lowest capacitance we can get with the varicaps is 9pF/2 (because of the series connection), which is 4.5pF.  So we need to decrease XL1.

<figure>
    <img src="./doc/SSDRA_p34.png" alt="Colpitts Oscillator Schematic" width="400"/>
    <figcaption>Figure 2: Colpitts Oscillator Schematic, from "Solid State Design for the Radio Amateur", fig. 2.A from p.34 by Wes Hayward, W7ZOI</figcaption>
</figure>

## Expected frequency range
* f @ 1Vdc : ≤ 77MHz
* f @ 5Vdc : ≥ 97MHz
* Δf = 20MHz

## Inductor Selection
* SCA1812HR10GTT, C5182193, 100nH
* SCA1812HR15GTT, C5178936, 150nH

For a parallel RLC circuit, the quality factor is given by Q = R / X0, where X0 = ω * L or X0 = 1 / (ω * C).  So we get a higher Q with a smaller inductor and a larger capacitor.  See [here](https://youtu.be/He0-X6FCLMo?list=PL9Ox3wpnB0kqekAyz6blg4YdvoEMoJNJY) for more details.

## Varicap Selection
| Part number | Package | LCSC | C @ 1Vdc [pF] | C @ 4Vdc [pF] |  Capacitance ratio |
|-------------|---------|------|---------------|---------------|-----------------|
| BB535E7904HTSA1 | SOD323 | C3272477 | 18 | 10 | 1.8 |
| HVU202B | SOD323 |C475704 | 20 | 13 | 1.54 |
| BB133 | SOD323 |C5334579 | 35 | 18 | 1.94 |
| BB910 | SOD323 |C475628 | 35 | 18 | 1.94 |
| 1SV324TPH3F | SOD323 | C3272930 | 50 | 11 | 4.55 |
| BB 640 E6327 | SOD323 |C533692 | 70 | 40 | 1.75 |
| BB175 | SOD523 | C3272656 | 60 | 32 | 1.88 |
| 1SV323(TPH3,F) | SOD523 | C2847421 | 30 | 7 | 4.29 |
| 1SV325,H3F(T | SOD523 | C4153758 |50 | 12 | 4.17 |

Because two diodes in anti-series are used, the capacitance will be halved.  

Some configurations that might work:
* D2, D3 = BB133||BB133, L1=100nH → f = [75MHz@1Vdc → 95MHz@4Vdc]
  * with a little higher DC-offset we arrive at 97MHz.
  * 4 diodes are required to achieve the desired tuning range, which is not ideal.
* D2, D3 = 1SV323, L1=150nH, C2= 4pF → f = [76MHz@1Vdc → 98MHz@4Vdc]
  * diodes are SOD523, which might require a PCB redesign
  * L1 value is ok, but the 150nH inductor is a 4248 instead of a 5910, which will require a PCB redesign.
  * The tuning range has no margin, which is not ideal.
* D2, D3 = 1SV325, L1=100nH, C2= 8pF → f = [77MHz@1Vdc → 102MHz@4Vdc]
  * diodes are SOD523, which might require a PCB redesign
  * L1 value is ok
  * tuning range has some margin, which is good.

In [3]:
def calc_frequency(Cvar):
    C_total = Ceq + Cvar
    return 1 / (2 * math.pi * math.sqrt(L1 * C_total))

C_1Vdc = 35e-12
C_4Vdc = 18e-12
Ceq = 10e-12  # Equivalent capacitance in Farads of Cc and Cf in series
L1 = 100e-9
#Ceq = Ceq +8e-12

#C_1Vdc = C_1Vdc / 2  # 2 diodes in series, so capacitance is halved
#C_4Vdc = C_4Vdc / 2  # 2 diodes in series, so capacitance is halved
print(f"f_min with C_1Vdc: {calc_frequency(C_1Vdc) / 1e6:.2f} MHz")
print (f"f_max with C_4Vdc: {calc_frequency(C_4Vdc) / 1e6:.2f} MHz")


f_min with C_1Vdc: 75.03 MHz
f_max with C_4Vdc: 95.11 MHz



For L1=169nH, the series capacitance of the varicaps needs to change from 6.19 pF to 15.41 pF.  6pF is an acceptable minimum and the tuning range is about 9pF, which is acceptable as well, so the BB133 looks like a good choice.

# Amplifier and buffer stage

Because of the large drive level (7dBm) required for the mixer stage, we will use a three stage amplifier/buffer.  That's one extra stage compared to the ZL2CTM design, but the waveforms will be cleaner.

In [4]:
# Buffer amplifier biasing

# 3rd stage biasing (extra buffer compared to ZL2CTM's design)
Ie4 = 15e-3
Ve4 = Vcc / 2
Re4 = Ve4 / Ie4
print(f"Emitter Resistance Re4: {Re4:.2f} Ohms")

# 2nd stage biasing
Ie3 = 3e-3 
Ve3 = Ve4 + 0.7
Re3 = Ve3 / Ie3
print(f"Emitter Resistance Re3: {Re3:.2f} Ohms")

# First stage biasing
Ie2 = 1e-3
Ve2 = 1

Vrc = Vcc - Ve3 - 0.7
Irc = Ie3/hfe + Ie2
Rc2 = Vrc / Irc
print(f"Collector Resistance Rc2: {Rc2:.2f} Ohms")

Re2 = Ve2 / Ie2
print(f"Emitter Resistance Re2: {Re2:.2f} Ohms")

Vr1_2 = Vcc - Ve2 - 0.7
Ir1_2 = Ie2 / hfe * 11
R1_2 = Vr1_2 / Ir1_2
print(f"Resistance R1_2: {R1_2:.2f} Ohms")

Vr2_2 = Ve2 + 0.7
Ir2_2 = Ie2 / hfe * 10
R2_2 = Vr2_2 / Ir2_2
print(f"Resistance R2_2: {R2_2:.2f} Ohms")

Emitter Resistance Re4: 166.67 Ohms
Emitter Resistance Re3: 1066.67 Ohms
Collector Resistance Rc2: 1067.96 Ohms
Emitter Resistance Re2: 1000.00 Ohms
Resistance R1_2: 30000.00 Ohms
Resistance R2_2: 17000.00 Ohms
